<a href="https://colab.research.google.com/github/shubhasreeav/SAM-for-WSSS/blob/main/SAM_FLOODNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Jul  2 09:54:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [ ]:
!pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.7 MB/s eta 0:00:00


In [ ]:
!mkdir -p {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P {HOME}/weights

In [ ]:
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

/content/weights/sam_vit_h_4b8939.pth ; exist: True


In [ ]:
!mkdir -p {HOME}/data

!wget -q https://media.roboflow.com/notebooks/examples/dog.jpeg -P {HOME}/data
!wget -q https://media.roboflow.com/notebooks/examples/dog-2.jpeg -P {HOME}/data
!wget -q https://media.roboflow.com/notebooks/examples/dog-3.jpeg -P {HOME}/data
!wget -q https://media.roboflow.com/notebooks/examples/dog-4.jpeg -P {HOME}/data

In [ ]:
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

In [ ]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

In [ ]:
mask_generator = SamAutomaticMaskGenerator(sam)
mask_predictor = SamPredictor(sam)

In [ ]:
import numpy as np
import cv2
import numpy as np
import supervision as sv
from PIL import Image
import glob
import os

In [ ]:
#building
image_path='/content/drive/MyDrive/work3/y4_floodnet_test'
label_path='/content/drive/MyDrive/work3/prediction'
images=glob.glob(image_path+'/*.jpg')

for img in images:

  image_bgr = cv2.imread(img)
  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
  height, width, _ = image_rgb.shape
  mask_predictor.set_image(image_rgb)
  basenamef=os.path.basename(img)
  basename=basenamef.split('.')[0]
  label=label_path+'/'+basename+'.txt'


  lines=[]
  masks_all=[]
  scores_all=[]
  logits_all=[]
  with open(label) as f:
      for line in f:
        lines.append(line.strip())
  for i in range(len(lines)):
      elements = lines[i].split()  # Split by spaces or the appropriate delimiter
      if len(elements) >= 5:

        if int(elements[0])==1:
          [ x1,y1,x2,y2] = np.array([int(float(elements[1])*width), int(float(elements[2])*height), int(float(elements[3])*width), int(float(elements[4])*height)])
          box=np.array([int(elements[1]),int(elements[2]),int(elements[3]),int(elements[4])])
          masks, scores, logits = mask_predictor.predict(
        box=box,
        multimask_output=True)

          masks_all.append(masks)
          scores_all.append(scores)
          logits_all.append(logits)
      else:
          print(f"Line {i} does not have enough elements.")
  if len(masks_all)>0:
    combined_mask=np.zeros_like(masks_all[0])
    for i in range(len(masks_all)):
      combined_mask+=masks_all[i]
    mask_int = np.where(combined_mask, 255, 0).astype(np.uint8)



    full_mask=np.zeros_like((1080,1920))
    for channel in range(mask_int.shape[0]):
      # Extract each channel as a 2D array
      channel_data = mask_int[channel]

      # Create an image from the channel data
      channel_image = Image.fromarray(channel_data, mode='L')  # 'L' mode for grayscale

      # Define the file path to save the channel image
      file_path = f"/content/drive/MyDrive/work3/FloodNet/masks_building/channel/{basename}_mask_{channel}.png"  # Update the file path and format as needed

      # Save the channel image
      #channel_image.save(file_path)
      if channel==0:
        mask1=channel_data
      if channel==1:
        mask2=channel_data
      if channel==2:
        mask3=channel_data
    full_mask=np.zeros_like(mask1)
    combined_mask = np.logical_or(np.logical_or(mask1 == 255, mask2 == 255), mask3 == 255)

  # Assign 255 where the combined condition is True in 'full_mask'
    full_mask[combined_mask] = 255
    file_path = f"/content/drive/MyDrive/work3/building/{basename}.jpg"
    cv2.imwrite(file_path,full_mask)



In [ ]:
#vehicle
image_path='/content/drive/MyDrive/work3/y4_floodnet_test'
label_path='/content/drive/MyDrive/work3/prediction'
images=glob.glob(image_path+'/*.jpg')

for img in images:

  image_bgr = cv2.imread(img)
  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
  height, width, _ = image_rgb.shape
  mask_predictor.set_image(image_rgb)
  basenamef=os.path.basename(img)
  basename=basenamef.split('.')[0]
  label=label_path+'/'+basename+'.txt'


  lines=[]
  masks_all=[]
  scores_all=[]
  logits_all=[]
  with open(label) as f:
      for line in f:
        lines.append(line.strip())
  for i in range(len(lines)):
      elements = lines[i].split()  # Split by spaces or the appropriate delimiter
      if len(elements) >= 5:

        if int(elements[0])==5:
          [ xc,yc,w,h] = np.array([int(float(elements[1])*width), int(float(elements[2])*height), int(float(elements[3])*width), int(float(elements[4])*height)])
          box=np.array([int(elements[1]),int(elements[2]),int(elements[3]),int(elements[4])])
          masks, scores, logits = mask_predictor.predict(
        box=box,
        multimask_output=True)

          masks_all.append(masks)
          scores_all.append(scores)
          logits_all.append(logits)
      else:
          print(f"Line {i} does not have enough elements.")
  if len(masks_all)>0:
    combined_mask=np.zeros_like(masks_all[0])
    for i in range(len(masks_all)):
      combined_mask+=masks_all[i]
    mask_int = np.where(combined_mask, 255, 0).astype(np.uint8)



    full_mask=np.zeros_like((1080,1920))
    for channel in range(mask_int.shape[0]):
      # Extract each channel as a 2D array
      channel_data = mask_int[channel]

      # Create an image from the channel data
      channel_image = Image.fromarray(channel_data, mode='L')  # 'L' mode for grayscale

      # Define the file path to save the channel image
      file_path = f"/content/drive/MyDrive/work3/FloodNet/masks_vehicle/channel/{basename}_mask_{channel}.png"  # Update the file path and format as needed

      # Save the channel image
      #channel_image.save(file_path)
      if channel==0:
        mask1=channel_data
      if channel==1:
        mask2=channel_data
      if channel==2:
        mask3=channel_data
    full_mask=np.zeros_like(mask1)
    combined_mask = np.logical_or(np.logical_or(mask1 == 255, mask2 == 255), mask3 == 255)

  # Assign 255 where the combined condition is True in 'full_mask'
    full_mask[combined_mask] = 255
    file_path = f"/content/drive/MyDrive/work3/vehicle/{basename}.jpg"
    cv2.imwrite(file_path,full_mask)

In [ ]:
#tree
image_path='/content/drive/MyDrive/work3/y4_floodnet_test'
label_path='/content/drive/MyDrive/work3/prediction'
images=glob.glob(image_path+'/*.jpg')

for img in images:

  image_bgr = cv2.imread(img)
  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
  height, width, _ = image_rgb.shape
  mask_predictor.set_image(image_rgb)
  basenamef=os.path.basename(img)
  basename=basenamef.split('.')[0]
  label=label_path+'/'+basename+'.txt'


  lines=[]
  masks_all=[]
  scores_all=[]
  logits_all=[]
  with open(label) as f:
      for line in f:
        lines.append(line.strip())
  for i in range(len(lines)):
      elements = lines[i].split()  # Split by spaces or the appropriate delimiter
      if len(elements) >= 5:

        if int(elements[0])==2:
          [ xc,yc,w,h] = np.array([int(float(elements[1])*width), int(float(elements[2])*height), int(float(elements[3])*width), int(float(elements[4])*height)])
          box=np.array([int(elements[1]),int(elements[2]),int(elements[3]),int(elements[4])])
          masks, scores, logits = mask_predictor.predict(
        box=box,
        multimask_output=True)

          masks_all.append(masks)
          scores_all.append(scores)
          logits_all.append(logits)
      else:
          print(f"Line {i} does not have enough elements.")
  if len(masks_all)>0:
    combined_mask=np.zeros_like(masks_all[0])
    for i in range(len(masks_all)):
      combined_mask+=masks_all[i]
    mask_int = np.where(combined_mask, 255, 0).astype(np.uint8)



    full_mask=np.zeros_like((1080,1920))
    for channel in range(mask_int.shape[0]):
      # Extract each channel as a 2D array
      channel_data = mask_int[channel]

      # Create an image from the channel data
      channel_image = Image.fromarray(channel_data, mode='L')  # 'L' mode for grayscale

      # Define the file path to save the channel image
      file_path = f"/content/drive/MyDrive/work3/FloodNet/masks_tree/channel/{basename}_mask_{channel}.png"  # Update the file path and format as needed

      # Save the channel image
      #channel_image.save(file_path)
      if channel==0:
        mask1=channel_data
      if channel==1:
        mask2=channel_data
      if channel==2:
        mask3=channel_data
    full_mask=np.zeros_like(mask1)
    combined_mask = np.logical_or(np.logical_or(mask1 == 255, mask2 == 255), mask3 == 255)

  # Assign 255 where the combined condition is True in 'full_mask'
    full_mask[combined_mask] = 255
    file_path = f"/content/drive/MyDrive/work3/tree/{basename}.jpg"
    cv2.imwrite(file_path,full_mask)